In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import concurrent
from time import sleep
import folium
from folium import plugins

In [2]:
labels = ['municipio','estado', 'categoria', 'titulo', 'preco', 'href']
valores = []
li_list = []
# li_list2 = []
cep_dict = {}
soup_list = []
url = "http://sp.olx.com.br/sao-paulo-e-regiao/zona-leste/imoveis?o="
regx = re.compile('\d{5}-\d{3}')

In [3]:
def retornaSoupOLX(url, num):
    for iter in range(num):
        r = requests.get(str(url)+str(iter))
        soup = BeautifulSoup(r.content, 'html.parser')
        for ultag in soup.find_all('ul', {'class': 'list', 'id':'main-ad-list' }):
            for litag in ultag.find_all(lambda tag: tag.name == 'li' and tag.get('class') == ['item']):
                li_list.append(litag)

In [4]:
# def PopulaListaLi(soup):

In [5]:
def tiraEspecialString(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '')

In [6]:
def tiraEspecialFloat(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '').replace(',', '.')

In [7]:
%%time
def criaListaValores(li_list):
    for elements in li_list:
        x = BeautifulSoup(str(elements), 'html.parser')
        #encontra a parte onde estão as informaçoes no html
        info = x.find('div', {'class':'col-2'})
        #encontra a string onde estão o municipio e estado separados por virgula
        cidade_estado = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-region'}))
        #encontra a categoria do anuncio
        categoria = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-category'}))
        #encontra o titulo do anuncio
        titulo = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-1 mb5px'}).find('h3', {'class':'OLXad-list-title mb5px'}))
        #encontra o html do preço
        preco = x.find('div', {'class':'col-3'})
        
        #trata o html do preço verificando se existe ou não
        if preco.find('p', {'class':'OLXad-list-price'}):
            preco = tiraEspecialFloat(x.find('div', {'class':'col-3'}).find('p', {'class':'OLXad-list-price'}))
        else:
            preco = 0.0
        #retorna a tag href "link do anuncio"
        href=x.find('a', {'class':'OLXad-list-link'})
        
        #retorna o conteudo do atributo href na tag href
        href = href['href'].strip(' \t\n\r')
        #separa string  onde estão o municipio
        municipio = cidade_estado.split(',',1)[1].strip(' \t\n\r')
        #separa o estado da string onde estão o municipio
        estado = cidade_estado.split(',',1)[0].strip(' \t\n\r')
        categoria = categoria.strip(' \t\n\r')
        titulo = titulo.strip(' \t\n\r')
        valores.append([municipio, estado,categoria, titulo, preco, href])

Wall time: 0 ns


In [8]:
def findCEP(url):
    r = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(r.content, 'html.parser')
    cep = soup.find(lambda tag: tag.name == 'strong' and tag.get('class') == ['description'] and regx.match(tag.text.strip(' \t\n\r')) is not None)
    if cep:
        cep_dict.update({url:cep.text.strip(' \t\n\r')})
#         li_list2.append([url,cep.text.strip(' \t\n\r')])
    else:
        cep_dict.update({url:''})
#         li_list2.append('')

In [9]:
#encontrar url geral
#encontrar lis
#popular valores das lis
#criar df
#ver o head do df

In [10]:
%%time
soup = retornaSoupOLX(url, 100)

Wall time: 1min 5s


In [11]:
# PopulaListaLi(soup)

In [12]:
%%time
criaListaValores(li_list)

Wall time: 14.5 s


In [13]:
%%time
df = pd.DataFrame(valores, columns=labels)

Wall time: 2.5 ms


In [14]:
%%time
list_href = list(df['href'])

Wall time: 500 µs


In [15]:
%%time
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as e:
     e.map(findCEP, list_href)

Wall time: 5min 55s


In [16]:
len(cep_dict)

4949

In [17]:
%%time
df['cep'] = df['href'].map(cep_dict)

Wall time: 3 ms


In [18]:
%%time
df.to_excel('test_ddos.xlsx')

Wall time: 1.87 s


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
municipio    5000 non-null object
estado       5000 non-null object
categoria    5000 non-null object
titulo       5000 non-null object
preco        5000 non-null object
href         5000 non-null object
cep          4999 non-null object
dtypes: object(7)
memory usage: 273.5+ KB


In [22]:
df.head(10)

,municipio,estado,categoria,titulo,preco,href,cep
0,Belenzinho,"(São Paulo, Apartamentos)",Apartamentos,Camino Belem (ZL1244) Desocupado || Lazer Comp...,660000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03058-000
1,Belenzinho,"(São Paulo, Apartamentos)",Apartamentos,Rossi Lumina (ZL1243) Mobiliado || Lazer Completo,850000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03059-005
2,Mooca,"(São Paulo, Apartamentos)",Apartamentos,Apartamento mooca totalmente reformado ótima l...,2000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03121-020
3,Vila Bertioga,"(São Paulo, Casas)",Casas,"Sobrado Vila Bertioga,3 dormitorios 1 suite 3 ...",4000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03181-010
4,Vila Independência,"(São Paulo, Indústria e comércio)",Indústria e comércio,Loja física de AÇAÍ,35000,http://sp.olx.com.br/sao-paulo-e-regiao/indust...,04222-000
5,Mooca,"(São Paulo, Casas- Anúncio Profissional)",Casas- Anúncio Profissional,"Casa Mooca, 2 dormitórios, sala, cozinha, wc",1300,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03165-000
6,Vila Formosa,"(São Paulo, Apartamentos- Anúncio Profissional)",Apartamentos- Anúncio Profissional,Loft,400000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03357-050
7,Alto da Mooca,"(São Paulo, Indústria e comércio)",Indústria e comércio,Loja assistência técnica de celular,18000,http://sp.olx.com.br/sao-paulo-e-regiao/indust...,03176-001
8,Mooca,"(São Paulo, Apartamentos- Anúncio Profissional)",Apartamentos- Anúncio Profissional,Apartamento Padrão,899000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03119-001
9,Vila Carrão,"(São Paulo, Indústria e comércio- Anúncio Prof...",Indústria e comércio- Anúncio Profissional,"Sala comercial à venda, Vila Carrão, São Paulo",284000,http://sp.olx.com.br/sao-paulo-e-regiao/indust...,03402-001


In [15]:
df.municipio.unique

NameError: name 'df' is not defined

In [3]:
GOOGLE_MAPS_API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'

params = {
    'address': 'Belenzinho, São Paulo',
    'sensor': 'false',
    'region': 'br'
}
req = requests.get(GOOGLE_MAPS_API_URL, params=params)
res = req.json()

In [4]:
res['results'][0]['geometry']['location']['lat'], res['results'][0]['geometry']['location']['lng']

(-23.5408347, -46.59450959999999)

In [9]:
m = folium.Map([res['results'][0]['geometry']['location']['lat'], res['results'][0]['geometry']['location']['lng']], zoom_start=20)

In [13]:
# from folium.plugins import HeatMap
# HeatMap().add_to(m)